# Kerry Ronayne, DS3002 Project #2

In [1]:
#Import necssary packages for the code (if needed, install on device)
import os
import json
import pandas as pd
import pprint
import pymongo
from sqlalchemy import create_engine

In [50]:
#Set the necessary credentials-- the host_name to use for connection strings in later functitons, the different
### ports to refer from the jupyter notebook to the device versions of MongoDB and MySQl, the username and 
###password to access/make changes on Mongo and MySQL, as well as the namees for the database names of the
### created mongo and MySQL databasees

host_name = "localhost"
ports = {"mongo" : 27017, "mysql" : 3306}

user_id = "root"
pwd = "%"

src_dbname = "project_2" #what create in mongo
dst_dbname = "project_2_back2sql" #destination db

In [30]:
#Define useful functions from class notes and lecture:
## "get_sql_dataframe" connects jupyter to the MySQL databse, useful for verifying that a given database was in fact passed
### into MySQL and for later referencing the output of a given query (I used it in the report section to pass new
### queries through python in SQL language to generate general statistics about the data set)

## "get_mongo_dataframe" connects jupyter to MongoDB, extracting a collection or document from Mongo and converting it
### to a pandas dataframe to then manipulate within the python language and eventually send to SQL

## "set_dataframe" connects jupter again to MySQL, though this time uses the DF from the mongo function^^ to send
### that information to SQL into a database into new tables

def get_sql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe
def get_mongo_dataframe(user_id, pwd, host_name, port, db_name, collection, query):
    '''Create a connection to MongoDB, with or without authentication credentials'''
    if user_id and pwd:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db_name)
        client = pymongo.MongoClient(mongo_uri)
    else:
        conn_str = f"mongodb://{host_name}:{port}/"
        client = pymongo.MongoClient(conn_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query))) #passing in query
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    
    return dframe #gives back data from mongo db query in pandas DF

def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

### Mongo

In [44]:
#Populate the converted JSON file in Mongo by creating a client connection, loadining in the JSON file, and using the
## 'insert_many' function to then insert the loaded JSON file into Mongo.

#Dim Customers

conn_str = f"mongodb://{host_name}:{ports['mongo']}/"
client = pymongo.MongoClient(conn_str)
db = client[src_dbname]

customers = db.customers

with open('dim_customers.json') as openfile:
    file_data = json.load(openfile)

customers.insert_many(file_data)

client.close

<bound method MongoClient.close of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

In [45]:
#Dim Location

conn_str = f"mongodb://{host_name}:{ports['mongo']}/"
client = pymongo.MongoClient(conn_str)
db = client[src_dbname]

location = db.location

with open('dim_locations.json') as openfile:
    file_data = json.load(openfile)

location.insert_many(file_data)

client.close

<bound method MongoClient.close of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

In [46]:
#Dim Films
conn_str = f"mongodb://{host_name}:{ports['mongo']}/"
client = pymongo.MongoClient(conn_str)
db = client[src_dbname]

films = db.films

with open('dim_films.json') as openfile:
    file_data = json.load(openfile)

films.insert_many(file_data)

client.close

<bound method MongoClient.close of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

In [47]:
#Fact Table
conn_str = f"mongodb://{host_name}:{ports['mongo']}/"
client = pymongo.MongoClient(conn_str)
db = client[src_dbname]

facts = db.facts

with open('fact_table.json') as openfile:
    file_data = json.load(openfile)

facts.insert_many(file_data)

client.close

<bound method MongoClient.close of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

In [48]:
#Batches
conn_str = f"mongodb://{host_name}:{ports['mongo']}/"
client = pymongo.MongoClient(conn_str)
db = client[src_dbname]

batch1 = db.batch1
with open('batch1.json') as openfile:
    file_data = json.load(openfile)
batch1.insert_many(file_data)

batch2 = db.batch2
with open('batch2.json') as openfile:
    file_data = json.load(openfile)
batch2.insert_many(file_data)

batch3 = db.batch3
with open('batch3.json') as openfile:
    file_data = json.load(openfile)
batch3.insert_many(file_data)

batch4 = db.batch4
with open('batch4.json') as openfile:
    file_data = json.load(openfile)
batch4.insert_many(file_data)

batch5 = db.batch5
with open('batch5.json') as openfile:
    file_data = json.load(openfile)
batch5.insert_many(file_data)

client.close

<bound method MongoClient.close of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

# Turning Mongo to DF to send to SQL

In [51]:
#Create a new project-specific database in MySQL by creating a connection with the sql alchemy package. If it has
## already been created, the exception will note that.

try:
    exec_sql = f"CREATE DATABASE `{dst_dbname}`;"
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    sqlEngine.execute(exec_sql)
    sqlEngine.execute(f"USE {dst_dbname};")
except:
    print("This database already exists. Either drop the scheme in MySQL or move on to the next cell!")

## General Fact Table

In [56]:
# This does the same extracting from mongo as the previous cell, only doing so with the manual season collectiton
query = {}
port = ports["mongo"]
collection = "facts"

df_facts = get_mongo_dataframe(None, None, host_name, port, src_dbname, collection, query)
df_facts.head(2)

,facts_key,rental_key,inventory_key,title,release_year,length,rental_date,rental_duration,rental_rate,replacement_cost
0,606,4863.0,1,ACADEMY DINOSAUR,2006,86,2005-07-08 19:03:15,6,0.99,20.99
1,607,11433.0,1,ACADEMY DINOSAUR,2006,86,2005-08-02 20:13:10,6,0.99,20.99


In [57]:
#Sending the Pandas dataframes to mySQL using the 'set_dataframe' function in class.

dataframe = df_facts
table_name = 'fact_table_overview'
primary_key = 'facts_key'
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [59]:
#Verifying in SQL
sql_facts = "SELECT * FROM project_2_back2sql.fact_table_overview;"
df_facts = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_facts)
df_facts.head(2)

,facts_key,rental_key,inventory_key,title,release_year,length,rental_date,rental_duration,rental_rate,replacement_cost
0,606,4863.0,1,ACADEMY DINOSAUR,2006,86,2005-07-08 19:03:15,6,0.99,20.99
1,607,11433.0,1,ACADEMY DINOSAUR,2006,86,2005-08-02 20:13:10,6,0.99,20.99
